In [ ]:
import time
import numpy as np
import random
import pandas as pd

# Helper function to pad matrices to the next power of two
def pad_to_power_of_two(matrix, target_size):
    padded_matrix = np.zeros((target_size, target_size))
    original_size = matrix.shape[0]
    padded_matrix[:original_size, :original_size] = matrix
    return padded_matrix

# Helper function to remove padding
def remove_padding(matrix, original_size):
    return matrix[:original_size, :original_size]

# Winograd's Algorithm with padding support
def winograd(A, B):
    n = A.shape[0]
    # Create the array to hold results
    C = np.zeros((n, n))

    # Precompute the row and column factors
    row_factor = np.zeros(n)
    col_factor = np.zeros(n)

    for i in range(n):
        row_factor[i] = A[i, 0] + A[i, 1] if n > 1 else A[i, 0]
        col_factor[i] = B[0, i] + B[1, i] if n > 1 else B[0, i]

    for i in range(n):
        for j in range(n):
            C[i, j] = -row_factor[i] - col_factor[j]

    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]

    return C

# Strassen's Algorithm with padding support
def Strassen(A, B):
     n = A.shape[0]
     if n % 2 != 0:
        # If n is odd, add padding to make it even (for simplicity)
        A = np.pad(A, ((0, 1), (0, 1)), mode='constant')
        B = np.pad(B, ((0, 1), (0, 1)), mode='constant')
        n += 1

    # Divide matrices into quarters
     half = n // 2
     A11, A12, A21, A22 = A[:half, :half], A[:half, half:], A[half:, :half], A[half:, half:]
     B11, B12, B21, B22 = B[:half, :half], B[:half, half:], B[half:, :half], B[half:, half:]

    # Compute intermediary matrices
     M1 = (A11 + A22) @ (B11 + B22)
     M2 = (A21 + A22) @ B11
     M3 = A11 @ (B12 - B22)
     M4 = A22 @ (B21 - B11)
     M5 = (A11 + A12) @ B22
     M6 = (A21 - A11) @ (B11 + B12)
     M7 = (A12 - A22) @ (B21 + B22)

    # Combine intermediary matrices into result
     C11 = M1 + M4 - M5 + M7
     C12 = M3 + M5
     C21 = M2 + M4
     C22 = M1 - M2 + M3 + M6

    # Recombine submatrices
     C = np.vstack((np.hstack((C11, C12)), np.hstack((C21, C22))))
     return C[:A.shape[0], :A.shape[1]]  # Remove padding if added

# Tiled Matrix Multiplication with padding support
def tiled_matrix_multiplication(A, B, tile_size=2):
    n = A.shape[0]
    C = np.zeros((n, n))
    for i in range(0, n, tile_size):
        for j in range(0, n, tile_size):
            for k in range(0, n, tile_size):
                # Multiplying submatrices (tiles)
                C[i:i+tile_size, j:j+tile_size] += np.dot(
                    A[i:i+tile_size, k:k+tile_size],
                    B[k:k+tile_size, j:j+tile_size]
                )
    return C

# Environment and Agent setup as before...
class MatrixMultiplicationEnv:
    def __init__(self, A, B):
        self.A = A
        self.B = B

    def step(self, action):
        if action == 0:
            start_time = time.time()
            result = Strassen(self.A, self.B)
            time_taken = time.time() - start_time
        elif action == 1:
            start_time = time.time()
            result = winograd(self.A, self.B)
            time_taken = time.time() - start_time
        elif action == 2:
            start_time = time.time()
            result = tiled_matrix_multiplication(self.A, self.B, tile_size=2)
            time_taken = time.time() - start_time
        else:
            raise ValueError("Invalid action")
        reward = -time_taken  # Minimize time
        return reward, result

class QLearningAgent:
    def __init__(self, learning_rate=0.1, discount_factor=0.95, epsilon=0.1):
        self.q_table = {}  # State-action mapping
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon

    def get_action(self, state):
        if state not in self.q_table:
            self.q_table[state] = [0, 0]  # Initialize with zero values for both actions
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, 1)  # Explore
        else:
            return np.argmax(self.q_table[state])  # Exploit

    def update(self, state, action, reward, next_state):
        if next_state not in self.q_table:
            self.q_table[next_state] = [0, 0]
        best_next_action = np.argmax(self.q_table[next_state])
        td_target = reward + self.gamma * self.q_table[next_state][best_next_action]
        td_error = td_target - self.q_table[state][action]
        self.q_table[state][action] += self.lr * td_error


# Simulate the learning process
def train_rl_agent(A, B, episodes):
    env = MatrixMultiplicationEnv(A, B)
    agent = QLearningAgent()

    for episode in range(episodes):
        state = "matrix_multiplication"
        action = agent.get_action(state)
        reward, _ = env.step(action)
        next_state = "matrix_multiplication"
        agent.update(state, action, reward, next_state)
    return agent

# Get matrix size from the user
matrix_size = int(input("Enter the size of the matrix: "))

# Calculate the next power of two
next_power_of_two = 1 << (matrix_size - 1).bit_length()

# Generate random matrices and pad them if needed
A = np.random.randint(1, 10, (matrix_size, matrix_size))
B = np.random.randint(1, 10, (matrix_size, matrix_size))

A_padded = pad_to_power_of_two(A, next_power_of_two)
B_padded = pad_to_power_of_two(B, next_power_of_two)

# Train the agent
agent = train_rl_agent(A_padded, B_padded, episodes=4000)

# Test each algorithm and record time taken
env = MatrixMultiplicationEnv(A_padded, B_padded)
times = {}
action_names = {
    0: "Strassen's Algorithm",
    1: "Winograd Algorithm",
    2: "Tiled Matrix Multiplication"
}

for action in range(3):
    start_time = time.time()
    _, result_padded = env.step(action)
    result = remove_padding(result_padded, matrix_size)  # Remove padding
    time_taken = time.time() - start_time
    times[action_names[action]] = time_taken

# Display results in a table
df = pd.DataFrame(list(times.items()), columns=["Algorithm", "Time Taken (seconds)"])
print("\nTime taken by each algorithm:")
print(df)
print(A)
print(B)
# Check the agent's final action after training
state = "matrix_multiplication"
best_action = agent.get_action(state)
best_algorithm = action_names[best_action]
print(f"\nAgent's chosen action after training: {best_algorithm}")

reward, chosen_result_padded = env.step(best_action)
chosen_result = remove_padding(chosen_result_padded, matrix_size)
print("\nOutput Matrix using agent's chosen action:\n", chosen_result)

Enter the size of the matrix: 6

Time taken by each algorithm:
                     Algorithm  Time Taken (seconds)
0         Strassen's Algorithm              0.000090
1           Winograd Algorithm              0.000401
2  Tiled Matrix Multiplication              0.000387
[[5 3 9 8 9 1]
 [3 5 1 4 8 7]
 [7 6 2 1 2 1]
 [6 5 5 9 4 1]
 [8 2 6 3 2 1]
 [3 4 2 5 6 3]]
[[6 6 8 2 9 2]
 [5 5 7 1 2 1]
 [9 2 7 5 8 7]
 [9 5 7 2 4 8]
 [6 3 6 6 9 7]
 [8 3 3 1 3 7]]

Agent's chosen action after training: Strassen's Algorithm

Output Matrix using agent's chosen action:
 [[260. 133. 237. 129. 239. 210.]
 [192. 110. 163.  79. 154. 155.]
 [119.  90. 134.  45. 116.  63.]
 [219. 131. 208.  85. 179. 159.]
 [159.  94. 156.  67. 157. 105.]
 [161.  94. 146.  69. 134. 127.]]
